# Encoder from https://github.com/ghallak/jpeg-python
removed the argparse and hardcoded the input and ouput filenames

In [ ]:
import argparse
import os
import sys
import math
import numpy as np
#from utils import *
import jpeg_utils
from scipy import fftpack
from PIL import Image
from jpeg_huffman import HuffmanTree

In [ ]:

def quantize(block, component, Q_strength):
    q = jpeg_utils.load_quantization_table(component) * Q_strength
    return (block / q).round().astype(np.int32) # replace / and round() by //, check type


def block_to_zigzag(block):
    return np.array([block[point] for point in jpeg_utils.zigzag_points(*block.shape)])


def dct_2d(image):
    return fftpack.dct(fftpack.dct(image.T, norm='ortho').T, norm='ortho')


def run_length_encode(arr):
    # determine where the sequence is ending prematurely
    last_nonzero = -1
    for i, elem in enumerate(arr):
        if elem != 0:
            last_nonzero = i

    # each symbol is a (RUNLENGTH, SIZE) tuple
    symbols = []

    # values are binary representations of array elements using SIZE bits
    values = []

    run_length = 0

    for i, elem in enumerate(arr):
        if i > last_nonzero:
            symbols.append((0, 0))
            values.append(jpeg_utils.int_to_binstr(0))
            break
        elif elem == 0 and run_length < 15:
            run_length += 1
        else:
            size = jpeg_utils.bits_required(elem)
            symbols.append((run_length, size))
            values.append(jpeg_utils.int_to_binstr(elem))
            run_length = 0
    return symbols, values


def write_to_file(filepath, dc, ac, blocks_count, image_cols, Q_strength, tables):
    """ Write the data to a file in a binary format. 
        It uses functions from the utils module to convert the integer values to strings of 0 and 1s,
        before writing to file.
        This is required as we want to control the size of the binary encoding. 
        Some values are encoded in less than a byte = 8 bits.
    """
    try:
        f = open(filepath, 'w')
    except FileNotFoundError as e:
        raise FileNotFoundError(
                "No such directory: {}".format(
                    os.path.dirname(filepath))) from e

    for table_name in ['dc_y', 'ac_y', 'dc_c', 'ac_c']:

        # 16 bits for 'table_size'
        f.write(jpeg_utils.uint_to_binstr(len(tables[table_name]), 16))

        for key, value in tables[table_name].items():
            if table_name in {'dc_y', 'dc_c'}:
                # 4 bits for the 'category'
                # 4 bits for 'code_length'
                # 'code_length' bits for 'huffman_code'
                f.write(jpeg_utils.uint_to_binstr(key, 4))
                f.write(jpeg_utils.uint_to_binstr(len(value), 4))
                f.write(value)
            else:
                # 4 bits for 'run_length'
                # 4 bits for 'size'
                # 8 bits for 'code_length'
                # 'code_length' bits for 'huffman_code'
                f.write(jpeg_utils.uint_to_binstr(key[0], 4))
                f.write(jpeg_utils.uint_to_binstr(key[1], 4))
                f.write(jpeg_utils.uint_to_binstr(len(value), 8))
                f.write(value)

    # 32 bits for 'blocks_count'
    f.write(jpeg_utils.uint_to_binstr(blocks_count, 32))
    # 16 bits for 'image_cols', to recover the correct size when uncompressing 
    f.write(jpeg_utils.uint_to_binstr(image_cols, 16))
    # Quantization strength encoded in 8 bits
    f.write(jpeg_utils.uint_to_binstr(Q_strength, 8))
    
    for b in range(blocks_count):
        for c in range(3):
            category = jpeg_utils.bits_required(dc[b, c])
            symbols, values = run_length_encode(ac[b, :, c])

            dc_table = tables['dc_y'] if c == 0 else tables['dc_c']
            ac_table = tables['ac_y'] if c == 0 else tables['ac_c']

            f.write(dc_table[category])
            f.write(jpeg_utils.int_to_binstr(dc[b, c]))

            for i in range(len(symbols)):
                f.write(ac_table[tuple(symbols[i])])
                f.write(values[i])
    f.close()



In [ ]:

def encode(input_file, output_file, Q_strength):

    image = Image.open(input_file)
    # Convert to the YCbCr format which is more efficient for compression
    ycbcr = image.convert('YCbCr')

    npmat = np.array(ycbcr, dtype=np.uint8)
    #npmat[npmat>248] = 248 # correct a small artefact appearing for pixel values close to 255 (turning black in the uncompress process)
    rows, cols = npmat.shape[0], npmat.shape[1]
    print(f'Original image size {rows}x{cols}.')
    # To compress it, the image is cut into blocks of size 8x8
    # If the image height or width is not a multiple of 8, we crop it
    crop = False
    if rows % 8 != 0:
        rows = rows // 8 * 8
        crop = True
    if cols % 8 != 0:
        cols = cols // 8 * 8
        crop = True
    if crop == True:
        print(f'Cropped image size {rows}x{cols}.')
        npmat = npmat[:rows,:cols,:]
    blocks_count = rows // 8 * cols // 8
    
    # dc is the top-left cell of the block, ac are all the other cells
    # dc is the constant value, at frequency zero of the DCT
    dc = np.empty((blocks_count, 3), dtype=np.int32)
    ac = np.empty((blocks_count, 63, 3), dtype=np.int32) # 64-1 coefficient from the DCT

    # Iterate over all the 8x8 blocks
    print('Iterate over all 8x8 blocks...')
    block_index = 0
    for i in range(0, rows, 8):
        for j in range(0, cols, 8):
            for k in range(3):
                # split 8x8 block and center the data range to zero
                # [0, 255] --> [-128, 127]
                block = npmat[i:i+8, j:j+8, k] - 128
                # 2D Discrete Cosine Transform
                dct_matrix = dct_2d(block)
                #print(np.max(dct_matrix))
                quant_matrix = quantize(dct_matrix, 'lum' if k == 0 else 'chrom', Q_strength)
                zz = block_to_zigzag(quant_matrix)
                # Separate the first DCT component (constant, dc)
                # from the others (oscillating ones, ac)
                dc[block_index, k] = zz[0]
                ac[block_index, :, k] = zz[1:]
            block_index += 1
    
    print('Huffman and run length coding...')
    H_DC_Y = HuffmanTree(np.vectorize(jpeg_utils.bits_required)(dc[:, 0]))
    H_DC_C = HuffmanTree(np.vectorize(jpeg_utils.bits_required)(dc[:, 1:].flat))
    H_AC_Y = HuffmanTree(
            jpeg_utils.flatten(run_length_encode(ac[i, :, 0])[0]
                    for i in range(blocks_count)))
    H_AC_C = HuffmanTree(
            jpeg_utils.flatten(run_length_encode(ac[i, :, j])[0]
                    for i in range(blocks_count) for j in [1, 2]))

    tables = {'dc_y': H_DC_Y.value_to_bitstring_table(),
              'ac_y': H_AC_Y.value_to_bitstring_table(),
              'dc_c': H_DC_C.value_to_bitstring_table(),
              'ac_c': H_AC_C.value_to_bitstring_table()}
    
    print('Writing to file...')
    write_to_file(output_file, dc, ac, blocks_count, cols//8, Q_strength, tables)
    
    input_size = os.stat(input_file).st_size
    output_size = os.stat(output_file).st_size
    print(f"""Image in {input_file} of size {input_size:,}
              compressed and saved as 
              {output_file} of size {output_size:,}.""")
    print(f'Compression ratio: {output_size/input_size:.3f}')

In [ ]:
input_file = "coffee.bmp"
output_file = "coffeecompressed.jpg"
#input_file = "astronaut.bmp"
#output_file = "astronautcompressed.jpg"

Q_strength = 16 # Quantization strength between 1 and 255
# the higher Q_strength, the more compressed is the image
encode(input_file, output_file, Q_strength)

In [ ]:
image = Image.open(input_file)
# Convert to the YCbCr format which is more efficient for compression
ycbcr = image.convert('YCbCr')

npmat = np.array(ycbcr, dtype=np.uint8)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(npmat)

In [ ]:
np.min(npmat), np.max(npmat)

In [ ]:
np.min(npmat-3), np.max(npmat-2)

In [ ]:
np.min(ycbcr-)